In [ ]:
import os
import numpy as np
import xarray as xr
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf

import geoviews_tools as gt

gv.extension('matplotlib')

In [ ]:

ntime = 3
cscratch_path = os.path.join('/', 'global', 'cscratch1', 'sd', 'twhilton')
cscratch_path = '/Users/tim/work/Data/SummenWRF/yatir/'
LHctl_d03 = gt.yatir_to_xarray(os.path.join(cscratch_path, 'LH_d03_yatirZ50.nc'),
                               varname='LH',
                               groupname='ctl',
                               timerange=ntime)
LHytr_d03 = gt.yatir_to_xarray(os.path.join(cscratch_path, 'LH_d03_yatirZ50.nc'),
                               varname='LH',
                               groupname='yatirZ050',
                               timerange=ntime)
LHctl_d02 = gt.yatir_to_xarray(os.path.join(cscratch_path, 'LH_d02_yatirZ50.nc'),
                               varname='LH',
                               groupname='ctl',
                               timerange=ntime)
LHytr_d02 = gt.yatir_to_xarray(os.path.join(cscratch_path, 'LH_d02_yatirZ50.nc'),
                               varname='LH',
                               groupname='yatirZ050',
                               timerange=ntime)

ntime = 720
HFXctl_d03 = gt.yatir_to_xarray(os.path.join(cscratch_path, 'HFX_d03_yatirZ50.nc'),
                                varname='HFX',
                                groupname='ctl',
                                timerange=ntime)
HFXytr_d03 = gt.yatir_to_xarray(os.path.join(cscratch_path, 'HFX_d03_yatirZ50.nc'),
                                varname='HFX',
                                groupname='yatirZ050',
                                timerange=ntime)
HFXctl_d02 = gt.yatir_to_xarray(os.path.join(cscratch_path, 'HFX_d02_yatirZ50.nc'),
                                varname='HFX',
                                groupname='ctl',
                                timerange=ntime)
HFXytr_d02 = gt.yatir_to_xarray(os.path.join(cscratch_path, 'HFX_d02_yatirZ50.nc'),
                                varname='HFX',
                                groupname='yatirZ050',
                                timerange=ntime)

In [ ]:
pad03 = 0.1
pad02 = 1.0
lon_d03 = LHctl_d03.lon.values
lat_d03 = LHctl_d03.lat.values
lon_d02 = LHctl_d02.lon.values
lat_d02 = LHctl_d02.lat.values

Plot the data using GeoViews with matplotlib.  The plot shows up (correctly) in the area surrounding [Yatir Forest](https://en.wikipedia.org/wiki/Yatir_Forest).

In [ ]:
hv.output(max_frames=ntime)
map_LH_ytr_d02, map_LH_ytr_d03 = gt.build_geoviews_comparison(LHytr_d02, LHytr_d03)
map_LH_ctl_d02, map_LH_ctl_d03 = gt.build_geoviews_comparison(LHctl_d02, LHytr_d03)

TODO: why no variable/group labels on this plot?  Perhaps has to do with nesting warning below.

In [ ]:
LHmaps = hv.Layout((map_LH_ctl_d02.collate(), 
                    map_LH_ytr_d02.collate(), 
                    map_LH_ctl_d03.collate(), 
                    map_LH_ytr_d03.collate()),
                  label='LH (Wm$^{-2}$)').cols(2)

In [ ]:
# LHmaps

TODO: figure out nesting warning below

In [ ]:
# # print(map_LH_ctl_d03) # + map_LH_ytr_d03
# # print(map_LH_ctl_d03.collate())
# print('1-----')
# print(gf.coastline * gv.Dataset(LHytr_d02).to(gv.QuadMesh, groupby='time'))
# print('2-----')
# print(hv.Overlay((gf.coastline, gv.Dataset(LHytr_d02).to(gv.QuadMesh, groupby='time'))))
# # print('3-----')
# # print(LHytr_d02)
# print('4-----')
# print(gv.Dataset(LHytr_d02).to(gv.QuadMesh, groupby='time'))

In [ ]:
# LHmaps = hv.Layout((map_LH_ctl_d02.collate(), 
#             map_LH_ytr_d02.collate(), 
#                 map_LH_ctl_d03.collate(), 
#     map_LH_ytr_d03.collate()),
#                     label='LH (Wm$^{-2}$)').cols(2)

# Sensible Heat #

Yatir sensible heat flux in late August 2015 was 368.8 to 434.4 W m$^{-2}$ ([Brugger et al 2018](https://doi.org/10.1007/s10546-018-0371-5). Table 3).  There was a strong diurnal cycle for both sites (forest and desert).  Note the figure units are K m s$^{-1}$.  Figure from [Banergee et al, 2018]([www.atmos-chem-phys.net/18/10025/2018/).
            ![Figure 3](./Banergee_etal_ACP_2018_fig3.pdf)

In [ ]:
HFXmean_ytr = HFXytr_d03.mean(dim=('x', 'y'))
HFXmean_ctl = HFXctl_d03.mean(dim=('x', 'y'))
hv.Curve(HFXmean_ytr).opts(ylim=(350, 440))

In [ ]:
import pandas as pd
import seaborn as sns
dscombined = xr.concat((HFXytr_d03, HFXctl_d03), dim=pd.Index(['Yatir', 'Control'], name='WRFrun'))
hvds = hv.Dataset(dscombined, kdims=['time', 'x', 'y', 'WRFrun'], vdims=['HFX', 'lat', 'lon'])

In [ ]:
HFXmean = dscombined.groupby('WRFrun').mean(dim=['x', 'y'])
df = HFXmean.to_dataframe()
df.reset_index()
sns.set(rc={"figure.figsize": (12, 4)})
sns.lineplot(x='time', y='HFX', hue='WRFrun', style='WRFrun', data=df.reset_index())

In [ ]:
hv.QuadMesh(hvds.select(WRFrun='Control', time='2015-08-15T23:30:00'), ['lon', 'lat'])

In [ ]:
hv.QuadMesh(hvds.select(WRFrun='Control', time='2015-08-15T23:30:00'), ['lon', 'lat']) + hv.QuadMesh(hvds.select(WRFrun='Yatir', time='2015-08-15T23:30:00'), ['lon', 'lat'])